In [1]:
%run ../ipynb_util_tars.py

In [2]:
%env WANDB_PROJECT=thesis

env: WANDB_PROJECT=thesis


In [3]:
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# LLaMA3 8B

## Load ZO_UP data

In [4]:
# select dataset
from enum import Enum

class DatasetType(Enum):
    """Enum for the dataset type."""

    """Zora + OSDG upsampled dataset"""
    ZO_UP = "zo_up"
    """SwissText Shared Task 1 dataset (Zurich NLP)"""
    SWISSTEXT_SHARED_TASK1 = "swisstext_shared_task1"


DATASET_TYPE = DatasetType.ZO_UP

In [5]:
from datasets import load_dataset, Features, Value, ClassLabel
import pickle

# load the dataset
# note: if you don't have the data in the folder, use the download-data.sh script

match DATASET_TYPE:
    case DatasetType.ZO_UP:
        # dont need to use manual features as class_encode_column will create ClassLabel
        # careful: watch out for the order of the ClassLabel as it doesn't map directly to the SDG class. need use mapping functions (id2label, label2id)
        # sdgs = [str(i) for i in range(1, 18)] + ["non-relevant"]
        # features = Features({"sdg": ClassLabel(num_classes=len(sdgs), names=sdgs), "abstract": Value("string")})

        dataset = load_dataset("csv", data_files=str(DATA_DIR_PATH / "zo_up.csv"))
        dataset = dataset.rename_columns({"sdg": "SDG", "abstract": "ABSTRACT"}).class_encode_column("SDG")
        dataset = dataset["train"].train_test_split(test_size=0.3, stratify_by_column="SDG", seed=SEED)
    case DatasetType.SWISSTEXT_SHARED_TASK1:
        dataset = load_dataset("json", data_files=str(DATA_DIR_PATH / "swisstext-2024-sharedtask" / "task1-train.jsonl"))
        dataset = dataset["train"].train_test_split(test_size=0.3, seed=SEED)

print(dataset["train"].features)
example = dataset["train"][0]
print("Example instance:\t", example)


# Label encodings / mappings
labels = set(dataset["train"]["SDG"])
id2label = {i: dataset["train"].features["SDG"].int2str(i) for i in range(len(labels))}
label2id = {dataset["train"].features["SDG"].int2str(i): i for i in range(len(labels))}

# save the encodings to a file for later use
ENCODING_DIR = BASE_DIR_PATH / "encodings" / DATASET_TYPE.value
# create the directory if it doesn't exist
ENCODING_DIR.mkdir(parents=True, exist_ok=True)
with open(ENCODING_DIR / "id2label.pkl", "wb") as f:
    pickle.dump(id2label, f)

with open(ENCODING_DIR / "label2id.pkl", "wb") as f:
    pickle.dump(label2id, f)

labels

{'SDG': ClassLabel(names=['1', '10', '11', '12', '13', '14', '15', '16', '17', '2', '3', '4', '5', '6', '7', '8', '9'], id=None), 'ABSTRACT': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'sdg_desc_short': Value(dtype='string', id=None), 'sdg_desc_long': Value(dtype='string', id=None)}
Example instance:	 {'SDG': 16, 'ABSTRACT': 'The first attempts to modernize simply replaced the single huge engine with a huge electric motor, changing little. The drive-shafts were replaced by wires, the huge steam engine by dozens of small motors. Factories spread out, there was natural light, and room to use ceiling-slung cranes. Workers had responsibility for their own machines, they needed better training and better pay. The electric motor was a wonderful invention, once we changed all the everyday details that surrounded it.', 'id': None, 'sdg_desc_short': None, 'sdg_desc_long': None}


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}

In [6]:
from transformers import LlamaTokenizer, AutoTokenizer

# whether the text should be lowered or not
SHOULD_LOWER = False

# base model
HF_MODEL_NAME = "meta-llama/Meta-Llama-3-8B"
# final model
MODEL_NAME = f"{HF_MODEL_NAME}-ft-{DATASET_TYPE.value}" + ("-lower" if SHOULD_LOWER else "")

tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
# print(tokenizer.pad_token, tokenizer.eos_token)


def preprocess_data(instances):
    match DATASET_TYPE:
        case DatasetType.SWISSTEXT_SHARED_TASK1:
            # take a batch of titles and abstracts and concat them
            titles = instances["TITLE"]
            abstracts = instances["ABSTRACT"]
            texts = [f"{title} {abstract}" for title, abstract in zip(titles, abstracts)]
        case DatasetType.ZO_UP:
            texts = instances["ABSTRACT"]

    if SHOULD_LOWER:
        texts = [text.lower() for text in texts]

    # encode
    encoding = tokenizer(texts, padding="longest", truncation=True, max_length=1024, return_tensors="pt")

    # add labels
    encoding["label"] = torch.tensor([label for label in instances["SDG"]])

    return encoding

encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)
encoded_dataset.set_format("torch")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <enum 'DatasetType'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <enum 'DatasetType'>: __main__.DatasetType has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


## Model

In [7]:
from transformers.data import DataCollatorWithPadding
from transformers.models.llama import LlamaForSequenceClassification
from peft import LoraConfig, TaskType, get_peft_model
from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    #bnb_4bit_use_double_quant=True,
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_compute_dtype=torch.bfloat16
)

model = LlamaForSequenceClassification.from_pretrained(
    HF_MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    num_labels=len(labels),
    token=HF_TOKEN
).bfloat16()
model.config.pad_token_id = tokenizer.pad_token_id
# model.resize_token_embeddings(len(tokenizer))

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


data_collator = DataCollatorWithPadding(tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 6,885,376 || all params: 7,511,879,680 || trainable%: 0.09165982807647952


In [8]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer, EvalPrediction

BATCH_SIZE = 4
METRIC_NAME = "accuracy"

args = TrainingArguments(
    f"{CHECKPOINT_PATH}/{MODEL_NAME}",
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    learning_rate=5e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=METRIC_NAME,
    seed=SEED,
    report_to="wandb",
    logging_steps=50,
)

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [9]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dvdblk (ngmi). Use `wandb login --relogin` to force relogin


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,4.305600,3.732876,0.099631,0.086774,0.099631,0.086003
100,3.561700,2.931734,0.195572,0.183408,0.195572,0.170282
150,2.772500,2.306216,0.284133,0.295728,0.284133,0.283620
200,1.642200,1.999870,0.391144,0.414634,0.391144,0.355445
250,1.482500,1.552569,0.560886,0.614577,0.560886,0.557449
300,1.111900,1.288644,0.649446,0.698016,0.649446,0.633108
350,0.620400,1.321636,0.697417,0.716792,0.697417,0.676932
400,0.422600,1.100049,0.704797,0.723166,0.704797,0.703862
450,0.440000,1.323731,0.686347,0.692328,0.686347,0.678333
500,0.356600,1.482628,0.708487,0.731960,0.708487,0.704965


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/user/dbielik/msc-thesis/venv/lib/python3.11/sit

TrainOutput(global_step=3160, training_loss=0.27387715019757236, metrics={'train_runtime': 8179.3468, 'train_samples_per_second': 1.54, 'train_steps_per_second': 0.386, 'total_flos': 3.834606101889024e+17, 'train_loss': 0.27387715019757236, 'epoch': 20.0})

In [10]:
trainer.evaluate()

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 1.444075107574463,
 'eval_accuracy': 0.7785977859778598,
 'eval_precision': 0.7841250633443292,
 'eval_recall': 0.7785977859778598,
 'eval_f1': 0.7735095272934409,
 'eval_runtime': 31.2633,
 'eval_samples_per_second': 8.668,
 'eval_steps_per_second': 2.175,
 'epoch': 20.0}

In [11]:
from sklearn.metrics import classification_report

model.eval()

# manual evaluation to show classifcation_report
true_labels = []
logits = []

for batch in encoded_dataset["test"]:
    batch = {k: v.to(trainer.args.device).unsqueeze(0) for k, v in batch.items()}
    label = batch.pop("label")

    # Forward pass
    with torch.no_grad():
        out = model(**batch)

    true_labels.append(label.item())
    logits.extend(out.logits.tolist())

probabilites = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
pred_labels = torch.argmax(probabilites, dim=-1).tolist()

report = classification_report(true_labels, pred_labels, target_names=[f"SDG {id2label[i]}" for i in range(len(labels))])

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbie

In [12]:
print(report)

              precision    recall  f1-score   support

       SDG 1       0.73      0.65      0.69        17
      SDG 10       0.47      0.53      0.50        17
      SDG 11       0.82      0.82      0.82        17
      SDG 12       0.69      0.53      0.60        17
      SDG 13       0.74      0.82      0.78        17
      SDG 14       0.94      0.94      0.94        17
      SDG 15       0.67      0.94      0.78        17
      SDG 16       0.67      0.59      0.62        17
      SDG 17       0.00      0.00      0.00         1
       SDG 2       0.76      0.81      0.79        16
       SDG 3       0.79      0.88      0.83        17
       SDG 4       0.94      0.88      0.91        17
       SDG 5       0.85      1.00      0.92        17
       SDG 6       0.80      0.94      0.86        17
       SDG 7       0.85      0.65      0.73        17
       SDG 8       0.80      0.50      0.62        16
       SDG 9       0.65      0.65      0.65        17

    accuracy              